# Project 2: Covid

In [3]:
# Import
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from pandas_profiling import ProfileReport
import os
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Literature

...

## 2. EDA, feature cleaning and engineering

### 2.1. Load data

In [4]:
# Load file with descriptions of data variables
data_description_folder = './data/'
data_description = pd.read_csv(data_description_folder + 'IDDO_SDTM_Data-Dictionary_v3.0_2022-10-06.csv', sep=';', encoding="ISO-8859-1")
data_description.columns = data_description.columns.str.replace(' ', '_')
data_description.head(3)

,Domain,Domain_Name,Variable_Name,Variable_Label,Variable_Type,Variable_Definition,Controlled_Terminology?
0,AU,Audiometry Test Results,STUDYID,Study Identifier,character,This variable contains the unique identifier f...,NaN
1,AU,Audiometry Test Results,DOMAIN,Domain Abbreviation,character,This variable contains the two-character abbre...,Y
2,AU,Audiometry Test Results,USUBJID,Unique Subject Identifier,character,This variable contains the unique subject iden...,NaN


In [3]:
# Load data
df_list = []
data_folder = './data/DATA_2022-09-01/'
print('Names of DataFrames ---> Description :')
for f in os.listdir(data_folder):
    if f != 'IN_2022-09-01.csv':
        df_name_str = f.split('_')[0]
        df_list.append(df_name_str)
        df_name = df_name_str
        locals()[df_name] = pd.read_csv(data_folder + f, sep=',', low_memory=False) 
        locals()[df_name].name = df_name_str
        print(df_name_str, '---> ', data_description.loc[data_description.Domain==df_name_str, 'Domain_Name'].iloc[0])
    else:
        #df_name_str = f.split('_')[0]
        #df_list.append(df_name_str)
        #df_name = df_name_str
        #mylist = []
        #for chunk in  pd.read_csv(data_folder + f, sep=',', low_memory=False, chunksize=2000):
            #mylist.append(chunk)
        #locals()[df_name] = pd.concat(mylist, axis= 0)
        #locals()[df_name].name = df_name_str
        #del mylist
        print('error')

Names of DataFrames ---> Description :
DM --->  Demographics
DS --->  Disposition
ER --->  Environmental Risk
HO --->  Healthcare Encounters
IE --->  Inclusion/Exclusion Criteria
error
LB --->  Laboratory Results
MB --->  Microbiology Specimen
PO --->  Pregnancy Outcomes
RELREC --->  Related Records
RP --->  Reproductive System Findings
RS --->  Disease Response and Clinical Classification
SA --->  Clinical and Adverse Events
SC --->  Subject Characteristics
SV --->  Subject Visits
TI --->  Trial Inclusion Exclusion Criteria
TS --->  Trial Summary
TV --->  Trial Visits
VS --->  Vital Signs


In [79]:
data_folder = './data/DATA_2022-09-01/'
LB = pd.read_csv(data_folder + 'LB_2022-09-01.csv', sep=',', low_memory=False) 
LB.name = 'LB'

### 2.2. Pre-selection of variables

#### For DM:

In [11]:
# See variables for DM
pd.set_option("display.max_colwidth", 1000)
data_description.loc[data_description.Domain==DM.name, ['Variable_Name', 'Variable_Type', 'Variable_Definition']]

,Variable_Name,Variable_Type,Variable_Definition
130,STUDYID,character,This variable contains the unique identifier for a study. This is the main key/identifier for all domains in the IDDO Data Repository  every domain table will have the STUDYID identifier.
131,DOMAIN,character,This variable contains the two-character abbreviation for the domain.
132,USUBJID,character,"This variable contains the unique subject identifier for a study. This is a secondary key/identifier for all subject-level domains in the IDDO Data Repository  every domain table containing subject-level information (i.e., all but the Trial Domains) will have the USUBJID identifier. This variable will identify unique subjects in the repository."
133,SUBJID,character,This variable contains the unique subject identifier provided by the data contributor.
134,RFSTDTC,character,"This variable describes the date and time of the start of the Subject Reference Period. The Subject Reference Period is defined by IDDO as starting with the subject's first study encounter and ending with the subject's final study encounter. RFSTDTC corresponds with the time and date of the subject's first study encounter (e.g., screening, enrollment, admission). This date will be used to calculate the relative days in the --DY?, --STDY, --ENDY variables. This date and time will be provided in ISO 8601 format. This variable will be blank for submissions that do not provide this initial date. All of the derived variables will also be blank since they are all calculated based on RFSTDTC."
135,DTHDTC,character,"This variable describes the date and time of the collection of the observation, administration of a test or collection of a specimen. This date and time will be provided in ISO 8601 format."
136,DTHFL,character,"This variable contains information about whether the subject died during the study period. The variable is expected to be null if the choice is not ""Yes""."
137,SITEID,character,This variable contains information about the study site.
138,INVID,character,This variable contains the unique investigator identifier of the data contributor. This may be used for COVID-19 data where many separate investigators have contributed data to a single large study.
139,INVNAM,character,This variable contains the clinical trial registry number associated with the subjects record.


In [12]:
# Combine AGE and AGETXT

print('Values for AGETXT: ', DM['AGETXT'].unique())
print('Min and Max for AGE: ', DM['AGE'].min(), DM['AGE'].max())

DM.loc[DM.AGETXT=='95+', 'AGE'] = 96. # We make the choice to put 96 for 95+

print('Min and Max for AGE after: ', DM['AGE'].min(), DM['AGE'].max())

Values for AGETXT:  [nan '95+']
Min and Max for AGE:  -70.0 94.089996
Min and Max for AGE:  -70.0 96.0


In [16]:
# Investigate and replace negative values for AGE

print('Values for AGEU: ', DM['AGEU'].unique())
print('Number of negative values for AGE: ', (DM['AGE']<0).sum())
print('Number of NA for AGE: ', DM['AGE'].isna().sum())

DM.loc[DM.AGE<0, 'AGE'] = np.nan # We make the choice to replace negative values by NA

print('Number of NA for AGE after: ', DM['AGE'].isna().sum())

Values for AGEU:  ['YEARS' nan 'MONTHS' 'DAYS']
Number of negative values for AGE:  8
Number of NA for AGE:  22493
Number of NA for AGE after:  22501


In [30]:
# Combine AGE and AGEU

print('Values for AGEU: ', DM['AGEU'].unique())
print('Number of NA for AGEU when AGE is not NA: ', DM.loc[DM.AGE.notna(), 'AGEU'].isna().sum())
print('Min and Max for AGE when AGEU is NA: ', DM.loc[DM.AGEU.isna(), 'AGE'].unique())
print('Min and Max for AGE: ', DM['AGE'].min(), DM['AGE'].max())

# We make the choice to keep the YEARS unit
DM.loc[DM.AGEU=='MONTHS', 'AGE'] /= 12
DM.loc[DM.AGEU=='DAYS', 'AGE'] /= 365

# TODO : Que fait-on quand NA pour AGEU ??

Values for AGEU:  ['YEARS' nan 'MONTHS' 'DAYS']
Number of NA for AGEU when AGE is not NA:  593
Min and Max for AGE when AGEU is NA:  [51. 59. 46. 74. 61. 70. 72. 50. 79. 42. 49. 38. 77. 78. 68. 58. 62. 53.
 56. nan 66. 71. 44. 52. 87. 80. 37. 41. 64. 67. 73. 27. 40. 63. 85. 88.
 91. 92. 60. 45. 76. 86. 83. 47. 57. 69. 54. 65. 55. 81. 29. 48. 43. 89.
 39. 75. 25. 36. 84. 28. 32. 33. 96. 90.  0. 24. 35. 23. 19.  3.  5. 34.
 22. 31. 20.  8. 17. 13.  4.  7. 12. 15. 16. 30. 18. 82.]
Min and Max for AGE:  0.0 96.0


In [32]:
# Investigate ARMCD and ARM
print('Values for ARMCD: ', DM['ARMCD'].unique())
print('Values for ARM: ', DM['ARM'].unique())

Values for ARMCD:  ['PER CLIN GUIDE']
Values for ARM:  ['Per Clinical Guidelines']


In [35]:
# Check the existence of columns in DM
DM.columns # We can see that DMDTC does not exist

Index(['STUDYID', 'DOMAIN', 'USUBJID', 'RFSTDTC', 'DTHFL', 'INVID', 'AGE',
       'AGETXT', 'AGEU', 'SEX', 'RACE', 'ETHNIC', 'ARMCD', 'ARM', 'COUNTRY',
       'DMDY'],
      dtype='object')

**=> To conclud, for DM, we can keep: USUBJID, DTHFL, AGE, SEX, RACE, ETHNIC, COUNTRY, DMDY:**

In [36]:
# Keep only some columns for DM
DM = DM[['USUBJID', 'DTHFL', 'AGE', 'SEX', 'RACE', 'ETHNIC', 'COUNTRY', 'DMDY']]
DM.columns

Index(['USUBJID', 'DTHFL', 'AGE', 'SEX', 'RACE', 'ETHNIC', 'COUNTRY', 'DMDY'], dtype='object')

In [42]:
DM.shape

(844451, 8)

#### For DS:

In [39]:
# See variables for DS
pd.set_option("display.max_colwidth", 1000)
data_description.loc[data_description.Domain==DS.name, ['Variable_Name', 'Variable_Type', 'Variable_Definition']]

,Variable_Name,Variable_Type,Variable_Definition
157,STUDYID,character,This variable contains the unique identifier for a study. This is the main key/identifier for all domains in the IDDO Data Repository  every domain table will have the STUDYID identifier.
158,DOMAIN,character,This variable contains the two-character abbreviation for the domain.
159,USUBJID,character,"This variable contains the unique subject identifier for a study. This is a secondary key/identifier for all subject-level domains in the IDDO Data Repository  every domain table containing subject-level information (i.e., all but the Trial Domains) will have the USUBJID identifier. This variable will identify unique subjects in the repository."
160,DSSEQ,number,"This variable contains a sequence number to ensure uniqueness of subject records within the domain. Each observation (each recorded as a separate row in the domain) will have a unique number within each subject, e.g., a subject with 10 observations will have 10 rows and each row is numbered sequentially from 1-10."
161,DSTERM,character,This variable contains the verbatim wording of the event as provided by the Data Contributor.
162,DSMODIFY,character,This variable contains a modification of the verbatim wording of the event. This is used to capture IDDO-defined standardised terms of the event.
163,DSDECOD,character,This variable contains a dictionary-derived text description of the event. This is defined by CDISC Controlled Terminology and IDDO Controlled Terminology. More details can be found in the IDDO Implementation Guide.
164,VISITNUM,number,This variable contains a number designating the planned clinical encounter number. This is a numeric version of the visit described in VISIT? and it is used for sorting.
165,VISIT,character,This variable contains the protocol-defined text description of the planned clinical encounter number (as defined in the Trial Visits (TV) Domain).
166,VISITDY,number,This variable contains a number designating the Study Day of the planned clinical encounter. This is also a numeric version of the visit described in VISIT? and can be used for sorting.


In [56]:
# Columns really in DS
DS.columns

Index(['STUDYID', 'DOMAIN', 'USUBJID', 'DSSEQ', 'DSTERM', 'DSMODIFY',
       'DSDECOD', 'VISITNUM', 'VISIT', 'VISITDY', 'DSDY', 'DSSTDY', 'DSEVINTX',
       'DSCDSTDY'],
      dtype='object')

In [40]:
# Investigate DSMODIFY
print('Values for DSMODIFY: ', DS['DSMODIFY'].unique())

Values for DSMODIFY:  [nan]


In [41]:
# Investigate DSTERM and DSDECOD
print('Values for DSTERM: ', DS['DSTERM'].unique())
print('Values for DSDECOD: ', DS['DSDECOD'].unique())
# We make the choice to keep only DSDECOD

Values for DSTERM:  ['DISCHARGED ALIVE' 'DEATH' 'TRANSFER TO OTHER FACILITY'
 'MISSING IN DATABASE' 'UNKNOWN' 'HOSPITALIZATION' 'PALLIATIVE DISCHARGE'
 'Death' 'Hospitalisation' 'Discharged alive' 'Transfer to other facility'
 'Medically fit for discharge (COVID-19 resolved) but remains in hospital for other reason'
 'Palliative discharge' 'Unknown'
 'Ongoing health care needs relating to this admission for COVID-19'
 'Ongoing health care needs NOT related to COVID episode'
 'Discharged alive expected to survive' 'DISCHARGED'
 'CURRENTLY HOSPITALISED' 'DISCHARGE' 'TRANSFERRED TO ANOTHER FACILITY'
 'ALIVE' 'DECEASED' 'Forwarding to home'
 'Transfer to another health care facility' 'Hospitalization'
 'Transfer to the Health District' 'Discharge against medical advice'
 'Palliative care' 'QUARANTINE CENTER'
 'TRANSFER TO OTHER HOSPITAL/FACILITY' 'LONG TERM CARE FACILITY'
 'DEATH IN HOSPITAL' 'TRANSFERRED TO ANOTHER UNIT' 'HOSPITAL DISCHARGE'
 'DISCHARGE WITH PALLIATIVE CARE' 'HOSPITALIZED

In [45]:
# Investigate VISITNUM, VISIT and VISITDY

print('Values for VISITNUM: ', DS['VISITNUM'].unique())
print('Values for VISIT: ', DS['VISIT'].unique())
print('Values for VISITDY: ', DS['VISITDY'].unique())

print('%NA for VISITNUM: ', DS['VISITNUM'].isna().sum()/DS.shape[0]*100)
print('%NA for VISIT: ', DS['VISIT'].isna().sum()/DS.shape[0]*100)
print('%NA for VISITDY: ', DS['VISITDY'].isna().sum()/DS.shape[0]*100)

Values for VISITNUM:  [nan  1.  2.]
Values for VISIT:  [nan 'Day 0' 'Week 2 Day 14']
Values for VISITDY:  [nan  1. 15.]
%NA for VISITNUM:  99.82092346266369
%NA for VISIT:  99.82092346266369
%NA for VISITDY:  99.82092346266369


In [46]:
# Investigate DSDY, DSEVINTX and DSCDSTDY

print('Values for DSDY: ', DS['DSDY'].unique())
print('Values for DSEVINTX: ', DS['DSEVINTX'].unique())
print('Values for DSCDSTDY: ', DS['DSCDSTDY'].unique())

print('%NA for DSDY: ', DS['DSDY'].isna().sum()/DS.shape[0]*100)
print('%NA for DSEVINTX: ', DS['DSEVINTX'].isna().sum()/DS.shape[0]*100)
print('%NA for DSCDSTDY: ', DS['DSCDSTDY'].isna().sum()/DS.shape[0]*100)

Values for DSDY:  [nan 78. 75. 29. 28. 24. 17. 32. 25. 21. 22. 30. 50. 38. 33. 36. 31. 45.
 80. 35. 59. 63. 34. 27. 26. 39. 49. 53. 16. 19. 20. 37. 72. 46. 11. 40.
 23. 48. 42. 54. 14. 90. 41. 51. 13.  7. 12.]
Values for DSEVINTX:  [nan 'AT ANY TIME AFTER DISCHARGE']
Values for DSCDSTDY:  [ nan  16.   2.  20.   8.   5.   4.   1.   9.  14.  25.  51.  18.  19.
  24.  12.  55.   6.  13.  23.  28. 109.   7.  11.  21.   3.]
%NA for DSDY:  99.87887781936428
%NA for DSEVINTX:  99.83814002888019
%NA for DSCDSTDY:  99.99211917743611


**=> To conclud, for DS, we can keep: USUBJID, DSSEQ and DSDECOD:**

In [58]:
# Keep only some columns for DS
DS = DS[['USUBJID', 'DSSEQ', 'DSDECOD']]
DS.columns

Index(['USUBJID', 'DSDECOD'], dtype='object')

In [59]:
DS.shape

(824787, 2)

#### For ER:

In [61]:
# See variables for ER
pd.set_option("display.max_colwidth", 1000)
data_description.loc[data_description.Domain==ER.name, ['Variable_Name', 'Variable_Type', 'Variable_Definition']]

,Variable_Name,Variable_Type,Variable_Definition
180,STUDYID,character,This variable contains the unique identifier for a study. This is the main key/identifier for all domains in the IDDO Data Repository  every domain table will have the STUDYID identifier.
181,DOMAIN,character,This variable contains the two-character abbreviation for the domain.
182,USUBJID,character,"This variable contains the unique subject identifier for a study. This is a secondary key/identifier for all subject-level domains in the IDDO Data Repository  every domain table containing subject-level information (i.e., all but the Trial Domains) will have the USUBJID identifier. This variable will identify unique subjects in the repository."
183,ERSEQ,number,"This variable contains a sequence number to ensure uniqueness of subject records within the domain. Each observation (each recorded as a separate row in the domain) will have a unique number within each subject, e.g., a subject with 10 observations will have 10 rows and each row is numbered sequentially from 1-10."
184,ERTERM,character,This variable contains the verbatim wording of the event as provided by the Data Contributor.
185,ERMODIFY,character,This variable contains a modification of the verbatim wording of the event. This is used to capture IDDO-defined standardised terms of the event.
186,ERCAT,character,This variable contains a categorization of the observation.
187,ERSCAT,character,This variable contains a further categorization of the observation.
188,ERPRESP,character,"This variable identifies whether an observation was pre-specified on the CRF. Values are null for spontaneously reported events, i.e. those collected as free-text verbatim terms."
189,EROCCUR,character,This variable identifies whether or not a pre-specified event has occurred. Values are null for spontaneously reported events.


In [62]:
# Columns really in ER
ER.columns

Index(['STUDYID', 'DOMAIN', 'USUBJID', 'ERSEQ', 'ERTERM', 'ERCAT', 'ERPRESP',
       'EROCCUR', 'ERSTAT', 'ERREASND', 'VISITNUM', 'VISIT', 'VISITDY', 'ERDY',
       'ERSTDY', 'ERENDY', 'EREVINTX', 'ERCNTRY'],
      dtype='object')

In [64]:
# Investigate ERCAT
print('Values for ERCAT: ', ER['ERCAT'].unique())

Values for ERCAT:  ['COVID-19 RISK FACTOR']


**=> To conclud, for ER, we can keep nothing because there is no decod of the event.**

#### For HO:

In [69]:
# See variables for HO
pd.set_option("display.max_colwidth", 1000)
data_description.loc[data_description.Domain==HO.name, ['Variable_Name', 'Variable_Type', 'Variable_Definition']]

,Variable_Name,Variable_Type,Variable_Definition
208,STUDYID,character,This variable contains the unique identifier for a study. This is the main key/identifier for all domains in the IDDO Data Repository  every domain table will have the STUDYID identifier.
209,DOMAIN,character,This variable contains the two-character abbreviation for the domain.
210,USUBJID,character,"This variable contains the unique subject identifier for a study. This is a secondary key/identifier for all subject-level domains in the IDDO Data Repository  every domain table containing subject-level information (i.e., all but the Trial Domains) will have the USUBJID identifier. This variable will identify unique subjects in the repository."
211,HOSEQ,number,"This variable contains a sequence number to ensure uniqueness of subject records within the domain. Each observation (each recorded as a separate row in the domain) will have a unique number within each subject, e.g., a subject with 10 observations will have 10 rows and each row is numbered sequentially from 1-10."
212,HOREFID,character,"This variable contains an identifier to distinguish duplicate findings, collections or events occurring within the same time period where no other timing information is available. This variable is used to make the rows' timing details unique."
213,HOTERM,character,This variable contains the verbatim wording of the event as provided by the Data Contributor.
214,HODECOD,character,This variable contains a dictionary-derived text description of the event. This is defined by CDISC Controlled Terminology and IDDO Controlled Terminology. More details can be found in the IDDO Implementation Guide.
215,HOCAT,character,This variable contains a categorization of the observation.
216,HOPRESP,character,"This variable identifies whether an observation was pre-specified on the CRF. Values are null for spontaneously reported events, i.e. those collected as free-text verbatim terms."
217,HOOCCUR,character,This variable identifies whether or not a pre-specified event has occurred. Values are null for spontaneously reported events.


In [70]:
# Columns really in HO
HO.columns

Index(['STUDYID', 'DOMAIN', 'USUBJID', 'HOSEQ', 'HOREFID', 'HOTERM', 'HODECOD',
       'HOCAT', 'HOPRESP', 'HOOCCUR', 'HOSTAT', 'HOREASND', 'HOPATT',
       'VISITNUM', 'VISIT', 'VISITDY', 'HODY', 'HOSTDY', 'HOENDY', 'HODUR',
       'HOSTRF', 'HOEVINTX', 'HOCDSTDY', 'HODISOUT', 'SELFCARE', 'HOINDC'],
      dtype='object')

In [71]:
# Investigate HODECOD
print('Values for HODECOD: ', HO['HODECOD'].unique())

Values for HODECOD:  ['HOSPITAL' 'INTENSIVE CARE UNIT']


**=> To conclud, for HO, we can keep: USUBJID, HOSEQ and HODECOD:**

In [72]:
# Keep only some columns for HO
HO = HO[['USUBJID', 'HOSEQ', 'HODECOD']]
HO.columns

Index(['USUBJID', 'HOSEQ', 'HODECOD'], dtype='object')

In [73]:
HO.shape

(2336786, 3)

#### For IE:

In [75]:
# See variables for IE
pd.set_option("display.max_colwidth", 1000)
data_description.loc[data_description.Domain==IE.name, ['Variable_Name', 'Variable_Type', 'Variable_Definition']]

,Variable_Name,Variable_Type,Variable_Definition
239,STUDYID,character,This variable contains the unique identifier for a study. This is the main key/identifier for all domains in the IDDO Data Repository  every domain table will have the STUDYID identifier.
240,DOMAIN,character,This variable contains the two-character abbreviation for the domain.
241,USUBJID,character,"This variable contains the unique subject identifier for a study. This is a secondary key/identifier for all subject-level domains in the IDDO Data Repository  every domain table containing subject-level information (i.e., all but the Trial Domains) will have the USUBJID identifier. This variable will identify unique subjects in the repository."
242,IESEQ,number,"This variable contains a sequence number to ensure uniqueness of subject records within the domain. Each observation (each recorded as a separate row in the domain) will have a unique number within each subject, e.g., a subject with 10 observations will have 10 rows and each row is numbered sequentially from 1-10."
243,IETESTCD,character,This variable identifies the shortened code for the name of the test or examination performed.
244,IETEST,character,This variable identifies the name of the test or examination performed.
245,IECAT,character,This variable contains a categorization of the observation.
246,IEORRES,character,"This variable contains the result of the test or examination performed as provided by the data contributor. The original data can be either numericm e.g. ""503"" or string, e.g., ""Positive""."
247,IESTRESC,character,"This variable contains the converted, standardized result of the test or examination performed. The data can be either numeric, e.g. ""503"", or string, e.g. ""Positive"", and is stored as a string in the repository. The standard units and conversion formulas (as applicable) are described in detail in the IDDO Implementation Guide in the relevant sections about the variable --STRESU."
248,IESTAT,character,This variable contains information about the status of the observation  specifically that it was not completed when it was expected to have been. This column should be empty when there is a value in the --OCCUR (Events and Interventions Domains) or --ORRES (Findings Domains) variables.


In [76]:
# Columns really in IE
IE.columns

Index(['STUDYID', 'DOMAIN', 'USUBJID', 'IESEQ', 'IETESTCD', 'IETEST', 'IECAT',
       'IEORRES', 'IESTRESC', 'IESTAT', 'IEREASND', 'IEDY'],
      dtype='object')

**=> To conclud, for IE, we can keep: USUBJID, IESEQ, IETESTCD, IESTRESC:**

In [77]:
# Keep only some columns for IE
IE = IE[['USUBJID', 'IESEQ', 'IETESTCD', 'IESTRESC']]
IE.columns

Index(['USUBJID', 'IESEQ', 'IETESTCD', 'IESTRESC'], dtype='object')

In [78]:
IE.shape

(948185, 4)

#### For IN:

TODO

#### For LB:

In [80]:
# See variables for LB
pd.set_option("display.max_colwidth", 1000)
data_description.loc[data_description.Domain==LB.name, ['Variable_Name', 'Variable_Type', 'Variable_Definition']]

,Variable_Name,Variable_Type,Variable_Definition
317,STUDYID,character,This variable contains the unique identifier for a study. This is the main key/identifier for all domains in the IDDO Data Repository  every domain table will have the STUDYID identifier.
318,DOMAIN,character,This variable contains the two-character abbreviation for the domain.
319,USUBJID,character,"This variable contains the unique subject identifier for a study. This is a secondary key/identifier for all subject-level domains in the IDDO Data Repository  every domain table containing subject-level information (i.e., all but the Trial Domains) will have the USUBJID identifier. This variable will identify unique subjects in the repository."
320,LBSEQ,number,"This variable contains a sequence number to ensure uniqueness of subject records within the domain. Each observation (each recorded as a separate row in the domain) will have a unique number within each subject, e.g., a subject with 10 observations will have 10 rows and each row is numbered sequentially from 1-10."
321,LBREFID,character,"This variable contains an identifier to distinguish duplicate findings, collections or events occurring within the same time period where no other timing information is available. This variable is used to make the rows' timing details unique."
322,LBTESTCD,character,This variable identifies the shortened code for the name of the test or examination performed. This is defined by CDISC Controlled Terminology.
323,LBTEST,character,This variable identifies the name of the test or examination performed. This is defined by CDISC Controlled Terminology.
324,LBCAT,character,This variable contains a categorization of the observation.
325,LBSCAT,character,This variable contains a further categorization of the observation.
326,LBORRES,character,"This variable contains the result of the test or examination performed as provided by the data contributor. The original data can be either numericm e.g. ""503"" or string, e.g., ""Positive""."


In [81]:
# Columns really in LB
LB.columns

Index(['STUDYID', 'DOMAIN', 'USUBJID', 'LBSEQ', 'LBTESTCD', 'LBTEST', 'LBCAT',
       'LBSCAT', 'LBORRES', 'LBORRESU', 'LBSTRESC', 'LBSTRESN', 'LBSTRESU',
       'LBSTAT', 'LBREASND', 'LBSPEC', 'LBMETHOD', 'VISITNUM', 'VISIT',
       'VISITDY', 'LBDY', 'LBSTRF', 'LBEVINTX', 'LBCDSTDY'],
      dtype='object')

TODO : grouper valeur et unités ??

**=> To conclud, for LB, we can keep: USUBJID, LBSEQ, LBTESTCD, LBORRES, LBORRESU, LBSTRESC, LBSTRESN, LBSTRESU:**

In [82]:
# Keep only some columns for LB
LB = LB[['USUBJID', 'LBSEQ', 'LBTESTCD', 'LBORRES', 'LBORRESU', 'LBSTRESC', 'LBSTRESN', 'LBSTRESU']]
LB.columns

Index(['USUBJID', 'LBSEQ', 'LBTESTCD', 'LBORRES', 'LBORRESU', 'LBSTRESC',
       'LBSTRESN', 'LBSTRESU'],
      dtype='object')

In [83]:
LB.shape

(7330863, 8)

### 2.3 EDA

In [13]:
# CHOOSE YOUR DATAFRAME :
df_profile = DS

In [14]:
# See variables of the DataFrame
pd.set_option("display.max_colwidth", 1000)
df_profile_var = data_description.loc[data_description.Domain==df_profile.name, ['Variable_Name', 'Variable_Type', 'Variable_Definition']]
df_profile_var.to_csv("./eda/" + df_profile.name + "_variables-description.csv", index=False)
df_profile_var

,Variable_Name,Variable_Type,Variable_Definition
157,STUDYID,character,This variable contains the unique identifier for a study. This is the main key/identifier for all domains in the IDDO Data Repository  every domain table will have the STUDYID identifier.
158,DOMAIN,character,This variable contains the two-character abbreviation for the domain.
159,USUBJID,character,"This variable contains the unique subject identifier for a study. This is a secondary key/identifier for all subject-level domains in the IDDO Data Repository  every domain table containing subject-level information (i.e., all but the Trial Domains) will have the USUBJID identifier. This variable will identify unique subjects in the repository."
160,DSSEQ,number,"This variable contains a sequence number to ensure uniqueness of subject records within the domain. Each observation (each recorded as a separate row in the domain) will have a unique number within each subject, e.g., a subject with 10 observations will have 10 rows and each row is numbered sequentially from 1-10."
161,DSTERM,character,This variable contains the verbatim wording of the event as provided by the Data Contributor.
162,DSMODIFY,character,This variable contains a modification of the verbatim wording of the event. This is used to capture IDDO-defined standardised terms of the event.
163,DSDECOD,character,This variable contains a dictionary-derived text description of the event. This is defined by CDISC Controlled Terminology and IDDO Controlled Terminology. More details can be found in the IDDO Implementation Guide.
164,VISITNUM,number,This variable contains a number designating the planned clinical encounter number. This is a numeric version of the visit described in VISIT? and it is used for sorting.
165,VISIT,character,This variable contains the protocol-defined text description of the planned clinical encounter number (as defined in the Trial Visits (TV) Domain).
166,VISITDY,number,This variable contains a number designating the Study Day of the planned clinical encounter. This is also a numeric version of the visit described in VISIT? and can be used for sorting.


In [15]:
# EDA on the DataFrame
profile = ProfileReport(df_profile, title="Pandas Profiling Report") # Choose the dataframe you want
profile.to_widgets()
profile.to_file("./eda/" + df_profile.name + ".html") # Write the name of the dataframe

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\felic\anaconda3\envs\ada\lib\site-packages\pandas_profiling\model\correlations.py:67: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  warnings.warn(
C:\Users\felic\anaconda3\envs\ada\lib\site-packages\scipy\stats\_stats_py.py:5218: RuntimeWarning: overflow encountered in longlong_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

#### Comments:

##### DM :

...